In [ ]:
import sys
sys.path.append('..')

from src.data.volume_dataloader import CTScanDataModule, CTDataSet
from src.models.unet import UNet
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from torch import sigmoid, where, cuda
import src.utils.ct_utils as ct_utils

DATA_PATH = '/home/gaetano/utooth/DATA/'

In [ ]:
dataset = CTScanDataModule(DATA_PATH, batch_size=5)

model = UNet(in_channels=1,
             out_channels=4,
             n_blocks=4,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=3,
             attention=False,
             loss_alpha=.5236,  # .7
             loss_gamma=1.0, # 3/4
             learning_rate=2e-3)

lr_monitor = LearningRateMonitor(logging_interval='epoch')
checkpoint = ModelCheckpoint(monitor='val_loss', dirpath='../outputs/checkpoints')
wandb_logger = WandbLogger(save_dir='../outputs/logs')
trainer = Trainer(accelerator='gpu',
                  devices=1,
                  strategy='auto',
                  log_every_n_steps=1,
                    max_epochs=2,
                #   max_epochs=50,
                  callbacks=[checkpoint],
                  logger=wandb_logger,
                  default_root_dir='../outputs')

In [6]:
# cuda.empty_cache()
trainer.fit(model, dataset)
# checkpoint.save_checkpoint(trainer)

/home/gaetano/utooth/utooth_env/lib/python3.10/site-packages/torch/cuda/__init__.py:145: UserWarning: 
NVIDIA GeForce RTX 3080 Ti Laptop GPU with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA GeForce RTX 3080 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/gaetano/.netrc
wandb: Currently logged in as: playweird to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



  | Name       | Type       | Params
------------------------------------------
0 | down_convs | ModuleList | 3.5 M 
1 | up_convs   | ModuleList | 2.1 M 
2 | conv_final | Conv3d     | 132   
------------------------------------------
5.6 M     Trainable params
0         Non-trainable params
5.6 M     Total params
22.412    Total estimated model params size (MB)
wandb: WARNING The project_name method is deprecated and will be removed in a future release. Please use `run.project` instead.


Validation sanity check: 0it [00:00, ?it/s]

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [1]:
check_model_path = checkpoint.best_model_path
model.eval()
x, y = dataset.ct_val[0]
y_hat = sigmoid(model(x.unsqueeze(0)))
x = x.squeeze(0).numpy()
y_hat = y_hat.squeeze(0)
y_hat = where(y_hat > 0.5, 1, 0).numpy()
y_hat = y_hat.sum(axis=0)

NameError: name 'checkpoint' is not defined

In [ ]:
%matplotlib tk
ct_utils.plot_3d_with_labels(x, y_hat, threshold=1200, transpose=[2, 1, 0], step_size=2)

In [ ]:
# trainer.tune(model, dataset)
# Run learning rate finder - Updated for PyTorch Lightning 2.0+
from pytorch_lightning.tuner import Tuner

tuner = Tuner(trainer)
lr_finder = tuner.lr_find(model, dataset)

# Results can be found in
lr_finder.results

# Plot with
fig = lr_finder.plot(suggest=True)
fig.show()

# Pick point based on plot, or get suggestion
new_lr = lr_finder.suggestion()

# update hparams of the model
model.hparams.learning_rate = new_lr